In [146]:
import gspread
import pandas as pd
import plotly
#import plotly.plotly as py

from speakerAnalysis import speakerAnalysisProject

from importlib import reload
reload(speakerAnalysisProject)

<module 'speakerAnalysis.speakerAnalysisProject' from '/home/seth/analysis/EDI/speakerAnalysis/speakerAnalysisProject.py'>

In [125]:
key = speakerAnalysisProject.readkeyfile('.keys')

In [126]:
# Use gspread api to access spreadsheet
gc = gspread.api_key(key[0])

sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/10vC4mm2HWrlcGIY72Or04t9A3uDUTKhudUAIyPmkF8o/edit?gid=0#gid=0")

print(sh.sheet1.get('A1'))

[['Speaker']]


In [127]:
# Convert spreadsheet into pandas data frame
data = sh.sheet1.get_all_values()
headers = data.pop(0)

df = speakerAnalysisProject.convertGStoPandas(data,headers)
print(df.columns)

Index(['Speaker', 'EDI member', 'Date', 'Type', 'CV?', 'UndergraduateName',
       'UndergraduateCategory', 'GraduateName', 'GraduateCategory',
       'Post-docName', 'Post-docCategory', 'InstitutionName',
       'InstitutionCategory', 'Private/Public', 'Ivy/Other'],
      dtype='object', name=0)


In [128]:
# Fraction of entries from an R1 instituion
notempty = df['InstitutionCategory']!=''
r1 = df['InstitutionCategory'] == 'R1'
print(sum(r1)/sum(notempty))

# Fraction of entries from an R1 undergraduate
notempty = df['Post-docCategory']!=''
r1 = df['Post-docCategory'] == 'R1'
print(sum(r1)/sum(notempty))

# Fraction of entries from an R1 graduate
notempty = df['GraduateCategory']!=''
r1 = df['GraduateCategory'] == 'R1'
print(sum(r1)/sum(notempty))

# Fraction of entries from an R1 undergraduate
notempty = df['UndergraduateCategory']!=''
r1 = df['UndergraduateCategory'] == 'R1'
print(sum(r1)/sum(notempty))

print(df['InstitutionCategory'].unique())

0.6923076923076923
0.5384615384615384
0.6538461538461539
0.5769230769230769
['R1' 'institute' 'special-focus' 'R1 ' 'international']


In [129]:
cat_cols=['UndergraduateCategory','GraduateCategory','Post-docCategory','InstitutionCategory']
# cats = []
# for i in range(len(cat_cols)):
#     cats.append(df[cat_cols[i]].unique())
#     print(cats[i])
    
for i in range(len(cat_cols)-1):
    if i==0:
        sourceTargetDf = df.groupby([cat_cols[i],cat_cols[i+1]]).size().reset_index().rename(columns={0:'count'})
        sourceTargetDf.columns = ['source','target','count']
        sourceTargetDf['source'] = cat_cols[i] + sourceTargetDf['source'].astype(str)
        sourceTargetDf['target'] = cat_cols[i+1] + sourceTargetDf['target'].astype(str)
    else:
        tempDf = df.groupby([cat_cols[i],cat_cols[i+1]]).size().reset_index().rename(columns={0:'count'})
        tempDf.columns = ['source','target','count']
        tempDf['source'] = cat_cols[i] + tempDf['source'].astype(str)
        tempDf['target'] = cat_cols[i+1] + tempDf['target'].astype(str)
        sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
    sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
    #print(sourceTargetDf)

temp = sourceTargetDf.groupby(['source','target']).size().reset_index().rename(columns={0:'count'})
temp = pd.concat([temp['source'],temp['target']])
print(list(dict.fromkeys(temp)))
# temp2 = df.groupby([cat_cols[1],cat_cols[2]]).size().reset_index().rename(columns={0:'count'})
# print(temp2)

['GraduateCategoryR1', 'GraduateCategoryR2', 'GraduateCategoryinternational', 'GraduateCategoryspecial-focus', 'Post-docCategoryNIH', 'Post-docCategoryR1', 'Post-docCategoryR1,NIH,R1,R1', 'Post-docCategoryinstitute', 'Post-docCategoryinternational', 'Post-docCategoryinternational; R1', 'Post-docCategoryintstitute; R1', 'Post-docCategoryspecial-focus', 'UndergraduateCategory?', 'UndergraduateCategoryR1', 'UndergraduateCategoryR2', 'UndergraduateCategoryinternational', 'UndergraduateCategoryliberal arts', 'UndergraduateCategoryno research', 'InstitutionCategoryR1', 'InstitutionCategoryinternational', 'InstitutionCategoryinstitute', 'InstitutionCategoryspecial-focus', 'InstitutionCategoryR1 ']


In [148]:
fig = speakerAnalysisProject.genSankey(df,cat_cols=['UndergraduateCategory','GraduateCategory','Post-docCategory','InstitutionCategory','Type'],title='Rearch categories')
plotly.offline.plot(fig, validate=False)

'temp-plot.html'

In [147]:
fig = speakerAnalysisProject.genSankey(df,cat_cols=['UndergraduateName','GraduateName','Post-docName','InstitutionName','Type'],title='Institution Names')
plotly.offline.plot(fig, validate=False)

'temp-plot.html'